**Imports**

In [1]:
import chess
import chess.pgn
import numpy as np
from aux_functions import *

**Importing the pgn data for training and transforming it to sparce matrices**

In [2]:
# Load pgn paths
pgns = import_data(1)
# Convert pgns to tensors
data = parse_pgn_to_tensors(pgns)

print(len(data))  # Number of games

NameError: name 'import_data' is not defined